# Aggregation

## 场景一

### (1) 描述

场景一中，所有实验按固定的模式运行，包含干扰循环和负载循环
1. 初始化 client 和 server(vm)
2. 启动 background 干扰，当前支持的干扰有: cpu/cache/mem/io/net
3. 启动 workload， 当前支持的workload有: redis/nginx/mysql
4. workload 结束后，记录一份 `workload_info`, 包含时间戳, 负载、干扰强度等metadata
5. 回到步骤3，执行下一个workload，当所有workload都执行完毕后，记录一份 `info_per_epoch`, 包含所有已经执行的 `workload_info`, 以及此次循环的干扰强度信息
6. 回到步骤2，直到所有的强度的干扰都指向完毕
7. 结束实验，记录实验的启动、结束时间，总共消耗的时间，以及干扰循环的总次数

以上信息保存在 `exp.json` 或 `date.json` 中，另外通过 Prometheus client 请求得到整个实验周期中的metric数据，保存在 `merged.csv` 中


### (2) 数据处理


#### 1. Quick Start


通过 `read_from_dir(dir)` 读取 metadata 和csv 数据创建 `ExpData` 实例

In [1]:
from aggregation import *
pd.set_option('display.max_rows', 10) 

exp_root = "/home/ict/appProfile/data/addtion_exp/standard_stress_cache_20231023055534/"
exp_data = read_from_dir(exp_root)

调用 `agg_epoch()` 方法之后，`exp_data` 会按每次干扰循环聚合数据, 这也是推荐的使用方法

In [2]:
df_epoch = exp_data.agg_epoch()
df_epoch

,stress_cache,host_cache_llc_capacity_numa_0,host_cache_llc_capacity_numa_1,host_cache_llc_capacity_numa_2,host_cache_llc_capacity_numa_3,host_kernel_context_switch,host_kernel_interrupt,host_kernel_thread_fork_fork,host_kernel_syscall_accept,host_kernel_syscall_accept4,...,host_kernel_syscall_sendmmsg,host_kernel_syscall_inotify_rm_watch,host_kernel_syscall_rmdir,host_kernel_syscall_statx,host_kernel_syscall_umount2,host_kernel_syscall_unlinkat,host_kernel_syscall_fallocate,host_kernel_syscall_getpgid,vm_hypervisor_emulator_syscall_count_fallocate,vm_hypervisor_emulator_syscall_duration_fallocate
redis_0,1,0.969117,0.932368,0.986070,0.977731,90670.274194,85881.451613,0.112903,0.564516,0.129032,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
redis_1,1,0.970935,0.945950,0.976326,0.976399,114668.628205,117440.076923,0.282051,0.576923,0.128205,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
redis_2,1,0.965691,0.944784,0.983974,0.976471,132115.717949,115039.717949,0.269231,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
redis_3,1,0.961199,0.938023,0.973161,0.978843,130271.881579,110798.842105,0.105263,0.526316,0.052632,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
redis_4,1,0.962216,0.938068,0.977273,0.975142,126330.525000,103445.037500,0.100000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
redis_4,32,0.969518,0.955313,0.990940,0.975814,65809.148649,70026.445946,0.378378,0.567568,0.027027,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
redis_5,32,0.960313,0.946970,0.988636,0.977445,71897.060606,79683.272727,0.136364,0.560606,0.090909,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
redis_6,32,0.953977,0.946378,0.983239,0.975355,79040.350000,89192.425000,0.150000,0.612500,0.125000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
redis_7,32,0.941974,0.945810,0.991335,0.970810,75544.512500,84729.450000,0.437500,0.600000,0.150000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


因为返回的仍然是一个 DataFrame, 因此后续可以根据数据处理的需要自行设置方法，如下展示一种从 `df_epoch` 数据中获取某个 workload 数据的流程

In [3]:
df_epoch_group = df_epoch.groupby(df_epoch.index)
keys = list(df_epoch_group.groups.keys())
keys

['redis_0',
 'redis_1',
 'redis_2',
 'redis_3',
 'redis_4',
 'redis_5',
 'redis_6',
 'redis_7',
 'redis_8']

使用 `groupby` 函数处理 Dataframe, 并从 `groups.keys()` 选择一个获取此 workload 的数据

In [4]:
df_workload = df_epoch_group.get_group(keys[0])
df_workload

,stress_cache,host_cache_llc_capacity_numa_0,host_cache_llc_capacity_numa_1,host_cache_llc_capacity_numa_2,host_cache_llc_capacity_numa_3,host_kernel_context_switch,host_kernel_interrupt,host_kernel_thread_fork_fork,host_kernel_syscall_accept,host_kernel_syscall_accept4,...,host_kernel_syscall_sendmmsg,host_kernel_syscall_inotify_rm_watch,host_kernel_syscall_rmdir,host_kernel_syscall_statx,host_kernel_syscall_umount2,host_kernel_syscall_unlinkat,host_kernel_syscall_fallocate,host_kernel_syscall_getpgid,vm_hypervisor_emulator_syscall_count_fallocate,vm_hypervisor_emulator_syscall_duration_fallocate
redis_0,1,0.969117,0.932368,0.986070,0.977731,90670.274194,85881.451613,0.112903,0.564516,0.129032,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
redis_0,2,0.964598,0.926792,0.994974,0.982517,87441.102564,84046.205128,0.128205,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
redis_0,4,0.974165,0.930842,0.986239,0.976385,88772.750000,85738.796875,0.140625,0.000000,0.031250,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
redis_0,8,0.971053,0.926904,0.987024,0.977196,85025.918919,84821.918919,0.270270,0.513514,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
redis_0,16,0.963618,0.945381,0.979472,0.968475,81288.870968,83684.274194,0.467742,0.580645,0.096774,...,0.0,0.241935,0.048387,0.032258,0.016129,0.032258,0.000000,0.0,0.000000,0.000000
redis_0,32,0.963710,0.946023,0.979839,0.973057,89208.370968,94168.435484,0.354839,0.612903,0.064516,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.016129,0.0,0.016129,1.548387


#### 2. Custom Process

`exp_data` 将读入的数据保存在 `exp` 与 `df` 两个字段中，可以通过 `exp_data.exp` 与 `exp_data.df` 来直接访问读入的数据， 如获取某个 workload info, 可以采取如下方式

In [5]:
workload_info = exp_data.exp["info_per_epoch"][0]["workloads"][keys[0]]
workload_info

{'start_time': 1698040546,
 'run_cmd': 'docker exec -it redis-client-1 memtier_benchmark  -s envoy --test-time 160 -t 1',
 'end_time': 1698040706,
 'name': 'redis_0',
 'stress': {'cache': {'cache': '1'}}}

随后，可以使用” `workload_df` 来获取此 workload 对应的 dataframe

In [6]:
exp_data.workload_df(workload_info)

,host_cache_llc_capacity_numa_0,host_cache_llc_capacity_numa_1,host_cache_llc_capacity_numa_2,host_cache_llc_capacity_numa_3,host_kernel_context_switch,host_kernel_interrupt,host_kernel_thread_fork_fork,host_kernel_syscall_accept,host_kernel_syscall_accept4,host_kernel_syscall_access,...,vm_block_io_bytes_io_write,vm_block_io_requests_write,vm_block_io_time_write,vm_block_io_flush_request_and_time_request_vda,vm_block_io_flush_request_and_time_time_vda,app_redis_qos_qps_of_redis_get,app_redis_qos_qps_of_redis_set,app_redis_qos_qps_of_redis_total,app_redis_qos_p99_latency_set,app_redis_qos_p99_latency_get
Time,,,,,,,,,,,,,,,,,,,,,
1698040550000,0.980114,0.951705,0.977273,0.988636,87495.0,83148.0,0.0,0.5,0.0,329.5,...,0,0.0,0.000000,0,0.0,39507.5,3951.5,43459.0,0.99,0.990000
1698040554000,0.974432,0.849432,0.991477,0.982955,88073.0,82286.0,0.0,0.5,0.0,329.5,...,0,0.0,0.000000,0,0.0,38915.0,3892.5,42807.5,0.99,0.990012
1698040558000,0.957386,0.948864,0.991477,0.980114,90041.5,84752.5,0.0,0.5,0.0,329.5,...,0,0.0,0.000000,0,0.0,39919.0,3991.0,43910.0,0.99,0.990026
1698040566000,0.982955,0.994318,0.971591,0.988636,88943.5,83581.5,0.0,0.5,0.0,329.5,...,0,0.0,0.000000,0,0.0,40468.0,4046.5,44514.5,0.99,0.990000
1698040570000,0.963068,0.906250,0.994318,0.963068,95489.0,89848.0,0.0,0.5,0.0,329.5,...,16384,3.5,0.003391,0,0.0,41526.5,4152.5,45679.0,0.99,0.990000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1698040690000,0.965909,0.960227,0.974432,0.982955,85645.5,82015.5,0.0,0.5,0.0,329.5,...,0,0.0,0.000000,0,0.0,39026.5,3901.5,42928.0,0.99,0.990000
1698040694000,0.982955,0.914773,1.005682,0.977273,92706.5,88314.5,0.0,1.0,1.0,330.5,...,0,0.0,0.000000,0,0.0,39406.5,3942.5,43349.0,0.99,0.990000
1698040698000,0.957386,0.940341,0.982955,0.957386,95659.5,89050.5,0.0,0.5,0.0,329.5,...,0,0.0,0.000000,0,0.0,40355.0,4032.5,44387.5,0.99,0.990012


默认情况下不会提取 "stress" 数据，可以指定参数 `with_stress` 来开启

In [7]:
exp_data.workload_df(workload_info, with_stress=True)

,stress_cache,host_cache_llc_capacity_numa_0,host_cache_llc_capacity_numa_1,host_cache_llc_capacity_numa_2,host_cache_llc_capacity_numa_3,host_kernel_context_switch,host_kernel_interrupt,host_kernel_thread_fork_fork,host_kernel_syscall_accept,host_kernel_syscall_accept4,...,vm_block_io_bytes_io_write,vm_block_io_requests_write,vm_block_io_time_write,vm_block_io_flush_request_and_time_request_vda,vm_block_io_flush_request_and_time_time_vda,app_redis_qos_qps_of_redis_get,app_redis_qos_qps_of_redis_set,app_redis_qos_qps_of_redis_total,app_redis_qos_p99_latency_set,app_redis_qos_p99_latency_get
Time,,,,,,,,,,,,,,,,,,,,,
1698040550000,1,0.980114,0.951705,0.977273,0.988636,87495.0,83148.0,0.0,0.5,0.0,...,0,0.0,0.000000,0,0.0,39507.5,3951.5,43459.0,0.99,0.990000
1698040554000,1,0.974432,0.849432,0.991477,0.982955,88073.0,82286.0,0.0,0.5,0.0,...,0,0.0,0.000000,0,0.0,38915.0,3892.5,42807.5,0.99,0.990012
1698040558000,1,0.957386,0.948864,0.991477,0.980114,90041.5,84752.5,0.0,0.5,0.0,...,0,0.0,0.000000,0,0.0,39919.0,3991.0,43910.0,0.99,0.990026
1698040566000,1,0.982955,0.994318,0.971591,0.988636,88943.5,83581.5,0.0,0.5,0.0,...,0,0.0,0.000000,0,0.0,40468.0,4046.5,44514.5,0.99,0.990000
1698040570000,1,0.963068,0.906250,0.994318,0.963068,95489.0,89848.0,0.0,0.5,0.0,...,16384,3.5,0.003391,0,0.0,41526.5,4152.5,45679.0,0.99,0.990000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1698040690000,1,0.965909,0.960227,0.974432,0.982955,85645.5,82015.5,0.0,0.5,0.0,...,0,0.0,0.000000,0,0.0,39026.5,3901.5,42928.0,0.99,0.990000
1698040694000,1,0.982955,0.914773,1.005682,0.977273,92706.5,88314.5,0.0,1.0,1.0,...,0,0.0,0.000000,0,0.0,39406.5,3942.5,43349.0,0.99,0.990000
1698040698000,1,0.957386,0.940341,0.982955,0.957386,95659.5,89050.5,0.0,0.5,0.0,...,0,0.0,0.000000,0,0.0,40355.0,4032.5,44387.5,0.99,0.990012


或者通过 `agg_one_workload` 获取此 workload 对应时间序列下的均值

In [8]:
exp_data.agg_one_workload(workload_info)

,host_cache_llc_capacity_numa_0,host_cache_llc_capacity_numa_1,host_cache_llc_capacity_numa_2,host_cache_llc_capacity_numa_3,host_kernel_context_switch,host_kernel_interrupt,host_kernel_thread_fork_fork,host_kernel_syscall_accept,host_kernel_syscall_accept4,host_kernel_syscall_access,...,vm_block_io_bytes_io_write,vm_block_io_requests_write,vm_block_io_time_write,vm_block_io_flush_request_and_time_request_vda,vm_block_io_flush_request_and_time_time_vda,app_redis_qos_qps_of_redis_get,app_redis_qos_qps_of_redis_set,app_redis_qos_qps_of_redis_total,app_redis_qos_p99_latency_set,app_redis_qos_p99_latency_get
0,0.969117,0.932368,0.98607,0.977731,90670.274194,85881.451613,0.112903,0.564516,0.129032,329.629032,...,1453.419355,0.290323,0.000142,0.064516,0.000014,40144.822581,4014.16129,44158.983871,0.99,0.990009


`exp_data` 也允许只获取某个 干扰循环 的数据，通过下标指示

In [9]:
exp_data.agg_epoch(0)

,stress_cache,host_cache_llc_capacity_numa_0,host_cache_llc_capacity_numa_1,host_cache_llc_capacity_numa_2,host_cache_llc_capacity_numa_3,host_kernel_context_switch,host_kernel_interrupt,host_kernel_thread_fork_fork,host_kernel_syscall_accept,host_kernel_syscall_accept4,...,host_kernel_syscall_bpf,host_kernel_syscall_fgetxattr,host_kernel_syscall_get_mempolicy,host_kernel_syscall_inotify_add_watch,host_kernel_syscall_mkdir,vm_hypervisor_emulator_syscall_count_ioctl,vm_hypervisor_emulator_syscall_count_newfstatat,vm_hypervisor_emulator_syscall_count_recvmsg,vm_hypervisor_emulator_syscall_count_sendmsg,vm_hypervisor_vcpu_syscall_count_write
redis_0,1,0.969117,0.932368,0.986070,0.977731,90670.274194,85881.451613,0.112903,0.564516,0.129032,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
redis_1,1,0.970935,0.945950,0.976326,0.976399,114668.628205,117440.076923,0.282051,0.576923,0.128205,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
redis_2,1,0.965691,0.944784,0.983974,0.976471,132115.717949,115039.717949,0.269231,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
redis_3,1,0.961199,0.938023,0.973161,0.978843,130271.881579,110798.842105,0.105263,0.526316,0.052632,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
redis_4,1,0.962216,0.938068,0.977273,0.975142,126330.525000,103445.037500,0.100000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
redis_5,1,0.951246,0.934751,0.982771,0.979747,111376.064516,68255.854839,0.145161,0.516129,0.032258,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
redis_6,1,0.926948,0.939448,0.986932,0.976705,80787.171429,61533.242857,0.657143,0.000000,0.057143,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
redis_7,1,0.921632,0.936045,0.980252,0.975055,48865.378049,45297.487805,0.097561,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
redis_8,1,0.935498,0.938985,0.985408,0.980695,37641.829545,36773.886364,0.102273,0.500011,0.000000,...,368.508383,4.500102,1.000023,2.000046,0.500011,1.000023,3.50008,302.506882,4.000091,0.500011


### 3. Advanced Usage

默认情况下 `exp_data` 对每个 workload 采用如下预处理手段，按顺序依次为:
1. `filter_column_startswith(col_prefix=("vm", "app"))`: 只选用 `vm`, `app` 前缀的指标
2. `filter_column_useless(std_min=1e-10)`: 过滤掉平均值为0, 或方差小于 `1e-10` 的指标
3. `filter_row_noise(col_prefix=("app"))`: 过滤行中 `app` 为前缀指标中的离群值

同时，对于每个 workload 采用如下聚合手段，按顺序依次为:
1. `lambda x : x.mean().to_frame().T`: 将一个workload时序数据按均值压缩为一行

`exp_data` 允许对上述处理进行自定义, 需要注意的是, 自定义的方法设置完毕之后，将会一直生效，包括在 `agg_epoch` 时
- `set_workload_preprocess_funcs(df_funcs):`: 自定义预处理手段
- `set_workload_agg_funcs(df_funcs):`: 自定义聚合手段

其中 `df_funcs` 为一组函数，每个函数都满足如下签名:
- `df_func(df: DataFrame) -> DataFrame`

In [10]:
exp_data.set_workload_preprocess_funcs([
    filter_column_startswith(col_prefix=("stress", "host", "vm", "app")),
    filter_column_useless(excol_prefix=("stress")),
])

exp_data.workload_df(workload_info, with_stress=True)

,stress_cache,host_cache_llc_capacity_numa_0,host_cache_llc_capacity_numa_1,host_cache_llc_capacity_numa_2,host_cache_llc_capacity_numa_3,host_kernel_context_switch,host_kernel_interrupt,host_kernel_thread_fork_fork,host_kernel_syscall_accept,host_kernel_syscall_accept4,...,vm_block_io_bytes_io_write,vm_block_io_requests_write,vm_block_io_time_write,vm_block_io_flush_request_and_time_request_vda,vm_block_io_flush_request_and_time_time_vda,app_redis_qos_qps_of_redis_get,app_redis_qos_qps_of_redis_set,app_redis_qos_qps_of_redis_total,app_redis_qos_p99_latency_set,app_redis_qos_p99_latency_get
Time,,,,,,,,,,,,,,,,,,,,,
1698040546000,1,0.968750,0.963068,0.968750,0.985795,103930.0,92411.0,0.0,0.5,0.0,...,6144,1.0,0.000171,1,0.000246,95102.0,9500.5,104602.5,2.512993,2.515383
1698040550000,1,0.980114,0.951705,0.977273,0.988636,87495.0,83148.0,0.0,0.5,0.0,...,0,0.0,0.000000,0,0.000000,39507.5,3951.5,43459.0,0.990000,0.990000
1698040554000,1,0.974432,0.849432,0.991477,0.982955,88073.0,82286.0,0.0,0.5,0.0,...,0,0.0,0.000000,0,0.000000,38915.0,3892.5,42807.5,0.990000,0.990012
1698040558000,1,0.957386,0.948864,0.991477,0.980114,90041.5,84752.5,0.0,0.5,0.0,...,0,0.0,0.000000,0,0.000000,39919.0,3991.0,43910.0,0.990000,0.990026
1698040562000,1,0.971591,0.838068,0.965909,0.991477,90490.0,85934.5,1.0,0.5,0.0,...,0,0.0,0.000000,0,0.000000,40900.5,4091.0,44991.5,0.990610,0.990549
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1698040690000,1,0.965909,0.960227,0.974432,0.982955,85645.5,82015.5,0.0,0.5,0.0,...,0,0.0,0.000000,0,0.000000,39026.5,3901.5,42928.0,0.990000,0.990000
1698040694000,1,0.982955,0.914773,1.005682,0.977273,92706.5,88314.5,0.0,1.0,1.0,...,0,0.0,0.000000,0,0.000000,39406.5,3942.5,43349.0,0.990000,0.990000
1698040698000,1,0.957386,0.940341,0.982955,0.957386,95659.5,89050.5,0.0,0.5,0.0,...,0,0.0,0.000000,0,0.000000,40355.0,4032.5,44387.5,0.990000,0.990012


In [11]:
# 获取最大值而不是平均值
exp_data.set_workload_agg_funcs([
    lambda x : x.max().to_frame().T,
])

exp_data.agg_one_workload(workload_info)

,host_cache_llc_capacity_numa_0,host_cache_llc_capacity_numa_1,host_cache_llc_capacity_numa_2,host_cache_llc_capacity_numa_3,host_kernel_context_switch,host_kernel_interrupt,host_kernel_thread_fork_fork,host_kernel_syscall_accept,host_kernel_syscall_accept4,host_kernel_syscall_access,...,vm_block_io_bytes_io_write,vm_block_io_requests_write,vm_block_io_time_write,vm_block_io_flush_request_and_time_request_vda,vm_block_io_flush_request_and_time_time_vda,app_redis_qos_qps_of_redis_get,app_redis_qos_qps_of_redis_set,app_redis_qos_qps_of_redis_total,app_redis_qos_p99_latency_set,app_redis_qos_p99_latency_get
0,0.985795,0.994318,1.017045,1.0,103930.0,92411.0,1.0,1.0,1.0,330.5,...,264192.0,3.5,0.003391,2.0,0.000512,95102.0,9500.5,104602.5,2.512993,2.515383
